In [1]:
import pyspark.sql
from pyspark.sql import SparkSession
from super_secret_password import PASSWORD

# PYSPARK INITALIZE
session = SparkSession.builder.appName('CLI User Interface Test').getOrCreate()

# TODO make a password saver or something

DATABASE_VALUES= {"url": "jdbc:mysql://localhost:3306/creditcard_capstone",
                "table":["CDW_SAPP_BRANCH", "CDW_SAPP_CREDIT_CARD", "CDW_SAPP_CUSTOMER"],
                "mode":"overwrite",
                "properties":{"user":"root", "password":PASSWORD, "driver":"com.mysql.cj.jdbc.Driver"}}


In [27]:


query = """
    SELECT *
    FROM cdw_sapp_customer
    JOIN cdw_sapp_credit_card 
    ON cdw_sapp_credit_card.CUST_SSN = cdw_sapp_customer.SSN
    WHERE CUST_ZIP = '17201' AND YEAR(TIMEID) = '2018' AND MONTH(TIMEID) = 'September'
    ORDER BY DAYOFMONTH(cdw_sapp_credit_card.TIMEID) DESC 
"""


# sql_db_df_custom = session.read.format("jdbc").options(
#     driver="com.mysql.cj.jdbc.Driver",
#     user="root",
#     password=PASSWORD,  
#     url=DATABASE_VALUES["url"],
#     dbtable=query).load()


In [23]:

sql_branch_df = session.read.jdbc(
        url=DATABASE_VALUES["url"],
        properties=DATABASE_VALUES["properties"],
        table=DATABASE_VALUES["table"][0]
    )

In [19]:
sql_credit_df = session.read.jdbc(
        url=DATABASE_VALUES["url"],
        properties=DATABASE_VALUES["properties"],
        table=DATABASE_VALUES["table"][1]
    )

In [20]:
sql_customer_df = session.read.jdbc(
        url=DATABASE_VALUES["url"],
        properties=DATABASE_VALUES["properties"],
        table=DATABASE_VALUES["table"][2]
    )

In [31]:
sql_credit_customer_df = sql_customer_df.join(sql_credit_df, sql_customer_df.SSN == sql_credit_df.CUST_SSN)

In [33]:
session.sql(query)

+----------------+---------+-------------+-------------------+------------+----------+--------+----------+---------+-------------------+-----------+---------+-------------------+-----------+----------------+---------+--------------+----------------+-----------------+--------+
|  CREDIT_CARD_NO|CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|  CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|       LAST_UPDATED|MIDDLE_NAME|      SSN|FULL_STREET_ADDRESS|BRANCH_CODE|  CREDIT_CARD_NO| CUST_SSN|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|  TIMEID|
+----------------+---------+-------------+-------------------+------------+----------+--------+----------+---------+-------------------+-----------+---------+-------------------+-----------+----------------+---------+--------------+----------------+-----------------+--------+
|4210653341381529|   Vienna|United States|CSavage@example.com|(555)1242572|        VA|   22180|     Clara|   Savage|2018-04-21 12:49:02|     lenore|123452490|Lincoln Str